# 🚀 Welcome to Your ADK Adventure - Tools & Memory! 🚀

Part 0: Setup & Authentication 🔑

In [2]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [3]:
api_key = getpass('Enter your Google API Key: ')

genai.configure(api_key=api_key)

os.environ['GOOGLE_API_KEY'] = api_key

genai.configure(api_key=api_key)
print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


In [12]:
# -- Agent Definition

# This function creates a new AI agent called "weekend_trip_agent"
def create_weekend_trip_agent():
  """Create the Spontaneous Day Trip Generator agent"""
  return Agent(
      name="weekend_trip_agent", # Internal name of the agent
      model="gemini-2.5-flash",  #The LLM powering the agent (fast + cheap)
      description="Agent specialized in generating spontaneous full-weekend itineraries based on mood, interests, and budget.",
      instruction="""
        You are the "Spontaneous Weekend Trip" Generator 🧳 – a specialized AI assistant that creates engaging full-weekend itineraries.

            Your Mission:
            Transform a simple mood or interest into a complete 2-day or 3-day weekend trip, with real-time details, while respecting a budget.

            Guidelines:
            1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
            2. **Weekend Structure**: Create plans for each day (e.g., Saturday and Sunday), dividing them into morning, afternoon, and evening.
            3. **Real-Time Focus**: Search for current operating hours and special events.
            4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN the itinerary in MARKDOWN FORMAT with day labels (e.g., **Friday**, **Saturday**, **Sunday**) and time blocks with specific venue names.
       """,
      tools=[google_search]
  )
weekend_trip_agent = create_weekend_trip_agent()
print(f"🧞 Agent '{weekend_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'weekend_trip_agent' is created and ready for adventure!


In [10]:
# --- A Helper Function to Run Our Agents ---
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.

session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"


🧠 Explanation: What does this function do?

This function is responsible for running a user query through a specific agent in a session and then showing the agent's final response nicely formatted in Markdown. It helps us see not only the final answer, but also how the agent "thinks" step by step.

🔑 Key Concepts:

🏃‍♀️ Runner
The Runner is the engine that manages the conversation between the user and the agent. It handles the entire flow: sending the question, tracking the steps the agent takes (called events), and collecting the final answer.

🔄 Async Event Loop
The runner.run_async(...) function returns a stream of asynchronous events.
Each event represents something the agent is doing — like thinking, calling a tool, or responding.
The line async for event in runner.run_async(...) loops through each of these events in real time, as they happen.

This lets us follow the agent’s reasoning step by step.
Think of it like watching the agent "work things out" before answering.

👀 is_router flag
This optional flag controls whether we want to print the internal steps (events) of the agent.

When is_router = False (default), we print each event — useful when learning or debugging.

When is_router = True, we hide those steps. This is useful when an agent is being used as a tool by another agent (like a sub-agent), and we don’t want to clutter the logs.

✅ Final Output
The agent eventually sends a special event called a final response.
When that happens, we extract just the final message and display it using Markdown (to look nice in notebooks).

🧪 Why is this useful?
This pattern is great for testing and learning how agents behave.
It gives us insight into the "thinking process" of the model, not just the end result.

In [13]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=weekend_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and interesting weekend trip in Venecia, Italy. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(weekend_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a relaxing and interesting weekend trip in Venecia, Italy. Keep it affordable!'

🚀 Running query for agent: 'weekend_trip_agent' in session: 'd0309717-86a8-49d9-bd75-7ddf8740df27'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Here's a relaxing and interesting, yet affordable, weekend trip to Venice, designed to immerse you in the city's unique charm without breaking the bank. This itinerary focuses on walking, free attractions, and local culinary experiences.

**Accommodation Tip:** To keep costs down, consider staying in areas slightly outside the main tourist hub, like Cannaregio or Castello, or even Mestre on the mainland, which offers more budget-friendly options with easy transport links to Venice.

---

### **Saturday: Grand Canals, Iconic Sights & Hidden Gems**

**Morning (9:00 AM - 1:00 PM): Piazza San Marco & Rialto Buzz**

*   **9:00 AM - 10:30 AM: Explore Piazza San Marco & St. Mark's Basilica Exterior**
    Begin your day early at the ico

Here's a relaxing and interesting, yet affordable, weekend trip to Venice, designed to immerse you in the city's unique charm without breaking the bank. This itinerary focuses on walking, free attractions, and local culinary experiences.

**Accommodation Tip:** To keep costs down, consider staying in areas slightly outside the main tourist hub, like Cannaregio or Castello, or even Mestre on the mainland, which offers more budget-friendly options with easy transport links to Venice.

---

### **Saturday: Grand Canals, Iconic Sights & Hidden Gems**

**Morning (9:00 AM - 1:00 PM): Piazza San Marco & Rialto Buzz**

*   **9:00 AM - 10:30 AM: Explore Piazza San Marco & St. Mark's Basilica Exterior**
    Begin your day early at the iconic Piazza San Marco to avoid the thickest crowds. Marvel at the grand architecture surrounding the square, including St. Mark's Basilica, Doge's Palace, and the Campanile. You can admire the stunning Byzantine architecture and gilded mosaics of St. Mark's Basilica from the outside for free. While entry to the main area of the Basilica is often free during non-service times, be aware that visiting specific parts like the museum or Pala d'Oro incurs a small fee (€3-€7).
*   **10:30 AM - 12:00 PM: Rialto Bridge & Rialto Market**
    Walk towards the Rialto Bridge, the oldest and most famous bridge spanning the Grand Canal. Enjoy the bustling atmosphere and panoramic views of the canal from its arch. Then, head to the nearby Rialto Market (Mercati di Rialto). Even if you're not buying, wandering through the vibrant fish and produce markets is a free and fascinating cultural experience.
*   **12:00 PM - 1:00 PM: Libreria Acqua Alta**
    Discover the whimsical Libreria Acqua Alta, a unique bookstore known for storing its books in bathtubs, boats, and even a gondola to protect them from flooding. It's a charming and free sight to behold.

**Lunch (1:00 PM - 2:00 PM): Affordable Venetian Bites**

*   **Cicchetti & Tramezzini Tour**
    Embrace the Venetian tradition of *cicchetti* (small, tapas-like snacks) and *ombre* (small glasses of wine). Head to a local *bacaro* (traditional Venetian wine bar) away from the main tourist areas, particularly in districts like Cannaregio or Dorsoduro. Prices typically range from €1 to €2 per cicchetto, and you can combine a few for a satisfying and authentic lunch. Look for *tramezzini* (crustless sandwiches) as another affordable option, often less than €3.

**Afternoon (2:00 PM - 6:00 PM): Dorsoduro's Art & Tranquility**

*   **2:00 PM - 4:00 PM: Explore Dorsoduro District & Squero San Trovaso**
    Wander through Dorsoduro, a quieter neighborhood known for its art galleries, charming cafes, and beautiful squares like Campo Santa Margherita. While here, stop by Squero San Trovaso, one of the last remaining traditional gondola repair yards, where you can watch craftsmen at work from across the canal for free.
*   **4:00 PM - 5:00 PM: Basilica di Santa Maria della Salute**
    Visit the stunning Basilica di Santa Maria della Salute, a magnificent Baroque church located at the entrance of the Grand Canal. Entry to the church is free, and its location offers fantastic views of the city and the lagoon.
*   **5:00 PM - 6:00 PM: Zattere Promenade & Gelato**
    Enjoy a leisurely stroll along the Zattere Promenade on the southern edge of Venice. This waterfront walk offers beautiful views of the Giudecca Canal, especially lovely as the afternoon light softens. Treat yourself to an affordable gelato from a local shop.

**Evening (7:00 PM onwards): Authentic Cannaregio Experience**

*   **7:00 PM - 9:00 PM: Cannaregio Exploration & Jewish Ghetto**
    Head to the Cannaregio district, considered one of the least touristy parts of Venice. Explore its charming canals and hidden churches. Visit the historic Jewish Ghetto, the oldest in the world, and wander its narrow streets. This area offers a glimpse into everyday Venetian life away from the crowds.
*   **9:00 PM onwards: Dinner at a Local Osteria**
    Find a traditional *osteria* or *trattoria* in Cannaregio for a reasonably priced dinner. Look for places popular with locals, often serving fresh seafood and classic Venetian dishes. Many of these establishments offer simple, homemade-style food that is both satisfying and affordable.

---

### **Sunday: Island Hues & Lagoon Views**

**Morning (9:00 AM - 1:00 PM): Murano's Glass Artistry**

*   **9:00 AM - 9:45 AM: Vaporetto to Murano**
    Purchase a 24-hour Vaporetto pass (€20) if you plan to visit both Murano and Burano today, as individual tickets can add up. Take Vaporetto Line 12 from Fondamente Nove to Murano. The journey takes about 39-45 minutes.
*   **9:45 AM - 12:00 PM: Free Glass Blowing Demonstration**
    Upon arrival in Murano, seek out a glass factory that offers free glass-blowing demonstrations. Many factories provide these without pressure to buy, giving you an fascinating insight into the centuries-old art. Some hotels can even arrange free transportation. Explore the glass shops, admiring the intricate works of art.
*   **12:00 PM - 1:00 PM: Explore Murano & Lunch**
    Stroll along Murano's canals, which are less crowded than Venice's center. For lunch, you can find more affordable eateries on Murano, or even pack a picnic from a supermarket in Venice to enjoy by the canal.

**Afternoon (1:00 PM - 5:00 PM): Burano's Colorful Charm**

*   **1:00 PM - 1:30 PM: Vaporetto to Burano**
    From Murano, take Vaporetto Line 12 directly to Burano. The journey is approximately 25 minutes.
*   **1:30 PM - 4:30 PM: Discover Burano's Colors & Lace**
    Burano is famous for its vibrantly colored houses, which make for stunning photographs. Wander through the picturesque fishing village, appreciating the unique color schemes of each home. You can also learn about Burano's traditional lace-making heritage, though demonstrations or purchases may incur costs. This island offers a peaceful and visually delightful experience.
*   **4:30 PM - 5:00 PM: Vaporetto back towards Venice**
    Take Vaporetto Line 12 back towards Fondamente Nove.

**Evening (6:00 PM onwards): Relaxed Farewell**

*   **6:00 PM - 7:00 PM: Gardens of Venice**
    Depending on your energy and location, consider a visit to one of Venice's green spaces, like the Royal Gardens near St. Mark's Square or the Papadopoli Gardens near Piazzale Roma. They offer a tranquil escape from the bustling city.
*   **7:00 PM onwards: Last Venetian Dinner**
    For your final evening, opt for another affordable and relaxed dinner. Perhaps try a *pizza al taglio* spot for a quick and satisfying slice (around €3-€5 per generous slice), or a pasta-to-go place (from €6). Enjoy a leisurely walk, getting "lost" in the quieter backstreets, which is one of the best free ways to experience the authentic Venice.

Enjoy your relaxing and interesting weekend in Venice!

--------------------------------------------------

